In [1]:
!pip install langchain
!pip install langchain-groq
!pip install langchain-community
!pip install beautifulsoup4


In [2]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='.env')

llm = ChatGroq(
    temperature = 0,
    groq_api_key = os.getenv("GROQ_API_KEY"),
    model_name = "llama-3.1-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [3]:
from langchain_community.document_loaders import WebBaseLoader #it extracts text from a url
loader = WebBaseLoader("https://careers.unitedhealthgroup.com/job/21090454/data-analyst-2-sas-remote-minnetonka-mn/")
page_data = loader.load().pop().page_content
print(page_data)












SAS Data Analyst 2 - Remote in Minnetonka, MN, US




















































 
 

Skip to content








My ProfileJoin Our Talent CommunityMy DashboardEnglishSpanish 










 Work HereCareer AreasMilitary & Veterans ProgramsOur CultureDiversity, Equity & InclusionWhat We OfferOur AffiliatesHiring Process & GuidanceAdviceEventsEarly CareersProgramsEventsProgramsBlogMy DashboardMy ProfileJoin Our Talent CommunityEnglishSpanishWork HereCareer AreasMilitary & Veterans ProgramsOur CultureDiversity, Equity & InclusionWhat We OfferOur AffiliatesHiring Process & GuidanceAdviceEventsEarly CareersProgramsEventsProgramsBlogMy DashboardMy ProfileJoin Our Talent CommunityEnglishSpanish 














Job Detailsnbowman2024-08-15T20:28:23+00:00 

2244831


View JobsView JobsView JobsView Jobs SAS Data Analyst 2 – Remote Requisition Number: 2244831 Job Category: Business & Data Analytics  Primary Location: Minnetonka, MN, US (Remote considered)  Apply Internal

In [4]:
from langchain_core.prompts import PromptTemplate

#for prompt extract
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in a JSON format containing the following keys:
    'role','experience','skills', and description'.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE)
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)



str

In [5]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'SAS Data Analyst 2',
 'experience': '3+ years',
 'skills': ['BASE SAS',
  'SAS/Macros',
  'SAS/SQL',
  'Big Data/Hadoop (Hive, Spark etc.)',
  'Python',
  'Power BI',
  'SQL',
  'Relational databases (Oracle, Teradata, DB2)',
  'MS Excel'],
 'description': 'Support the design, development, testing, and deployment of data analytics programs and processes for supporting various operational data stores using SAS and Relational Databases including Teradata, Snowflake and Oracle. Provide production support for scheduled and ad-hoc processes, address job failures, data issues and user concerns. Collect, interpret, and aggregate data from traditional and non-traditional data sources for supporting programs and applications utilizing various data analytics purposes. Understand data requirements and business needs to develop data tools such as dashboards and data visualizations using SAS and PowerBI. Use business intelligence, data visualization, query, analytic and statistical softwa

In [6]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [7]:
!pip install chromadb
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
  for _, row in df.iterrows(): #iter over dataframe rows
    collection.add(documents=row["Techstack"], #for each row, adding a document
                   metadatas={"links":row["Links"]},
                   ids = [str(uuid.uuid4())])

In [8]:
job = json_res
job['skills']

['BASE SAS',
 'SAS/Macros',
 'SAS/SQL',
 'Big Data/Hadoop (Hive, Spark etc.)',
 'Python',
 'Power BI',
 'SQL',
 'Relational databases (Oracle, Teradata, DB2)',
 'MS Excel']

In [9]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}]]

In [10]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
    You are Akash, a business development executive at Analytics. Analytics is an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools.
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
    process optimization, cost reduction, and heightened overall efficiency.
    Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Analytics
    in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase Analytics' portfolio: {link_list}
    Remember you are Akash, BDE at Analytics.
    Do not provide a preamble.
    ### EMAIL (NO_PREAMBLE):

    """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description":str(job), "link_list":links})
print(res.content)

Subject: Expert SAS Data Analyst Services for Enhanced Business Insights

Dear Hiring Manager,

I came across your job posting for a SAS Data Analyst 2 and was impressed by the role's focus on leveraging data analytics to drive business growth. As a Business Development Executive at Analytics, I'd like to introduce you to our team of experts who can help you achieve your data analytics goals.

At Analytics, we specialize in providing AI and software consulting services that empower businesses to make data-driven decisions. Our team of experienced SAS data analysts can support the design, development, testing, and deployment of data analytics programs and processes, ensuring seamless integration with your operational data stores.

Our expertise in SAS, Relational Databases (Oracle, Teradata, DB2), Big Data/Hadoop (Hive, Spark), and data visualization tools like Power BI makes us an ideal partner for your data analytics needs. We can help you collect, interpret, and aggregate data from v

In [11]:
!pip install streamlit -q

In [12]:
!wget -q -O - ipv4.icanhazip.com

34.138.56.139


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.138.56.139:8501

your url is: https://cuddly-goats-drive.loca.lt
USER_AGENT environment variable not set, consider setting it to identify your requests.
